## Function Closures

*Some snippets in this notebook are taken from BLG458E Functional Programming course slides which can be accessed through the following link: https://www.slideshare.net/uyar/tag/blg458e*


Functions have access to the variables defined outside the function.  

First let's have a look at an example in Python, which is more readable thus easier to understand :-)  

```Python
def set_step(step):
    def get_range(start, end):
        return list(range(start, end+1, step))
    return get_range```
   
As you see, get_range() function has access to the variable named *step* even if step is not defined in the get_range function. It is not a parameter passed to get_range() either.  

get_range() can access to step variable because of the outer *set_step* function.  

When we type the following;  
```Python 
step1 = set_step(1)```  
It returns a get_range() function whose step value is fixed (which is 1).  
So when we type the following;  
```Python
step1(3,10)```  
We get this list:  [3, 4, 5, 6, 7, 8, 9, 10]

Below is a Haskell implementation of the same function.


In [1]:
set_step :: Int -> (Int -> Int -> [Int])
set_step step = get_range
  where
    get_range :: Int -> Int -> [Int]
    get_range start end = [start, start+step .. end]
    
step1 :: (Int -> Int -> [Int])
step1 = set_step 1

step1 3 10

[3,4,5,6,7,8,9,10]

### @decorators

Decorator takes a function and returns a *decorated* version.

Let's see an example.

```Python
# info decorates function f and returns a wrapped version of it.
def info(f):
    def wrapped(x):
        print("Entering with parameter: {}".format(x))
        result = f(x)
        print("Exiting with result: {}".format(result))
        return result
    return wrapped

def fac(n):
	return 1 if n == 0 else n * fac(n-1)
```
Now when we type `info(fac)`, this returns a *wrapped* version of *fac* function which has additional features like showing what is passed and what is returned! The beauty of decorators is that you don't have to call info and get a wrapped version and so on. What you need to do is put `@info` on top of the function which you want to wrap. Here's the final version of the code:  

```Python
def info(f):
    def wrapped(x):
        print("Entering with parameter: {}".format(x))
        result = f(x)
        print("Exiting with result: {}".format(result))
        return result
    return wrapped

@info
def fac(n):
	return 1 if n == 0 else n * fac(n-1)
```
Now each time fac is called, it is wrapped by info function and gets all the new features defined in wrapped!  
When we type `fac(3)`, the following is displayed:  

*Entering with parameter: 3  
Entering with parameter: 2  
Entering with parameter: 1  
Entering with parameter: 0  
Exiting with result: 1  
Exiting with result: 1  
Exiting with result: 2  
Exiting with result: 6  
6*  

**Example 2**:

If you are familiar with dynamic programming, you probably know memoization which is basically remembering what is already calculated so as not to repeat same calculation steps over and over. Here's a memoized version of fibonacci number calculation which works way faster than simple version 

```Python
def memoize(f):
    cache = {}
    def wrapped(n):
        if n not in cache:
            cache[n] = f(n)
        return cache[n]
    return wrapped

@memoize
def fib(n):
    if n == 1 or n == 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)
```

Note that we do not modify fib function, it is the same as before.

### Partial Application
In Haskell, you can call a function with fewer parameters than it expects. What it does is it takes the parameters given, and then returns a new function which requires the remaining parameters. This is called *partial application*

In [2]:
-- the function that doubles every element in a list
doubleAll :: [Integer] -> [Integer]
doubleAll x = map (* 2) x

doubleAll [3,7,20]

[6,14,40]

**doubleAll** function makes use of partial application because multiplication function *(*)* expects 2 operands but is given one.  
This way, we can apply **"? * 2 "** operation to every element in the list via *map*.

It is not possible to apply partial application when the argument to which we want to apply the function is not the first argument.

#### Operators Sections
Operators can be applied partially, as shown in the previous example.  
* (op x) y = y op x  
* (op y) x = x op y

In [3]:
(+8) 50

58

In [4]:
(>2) 10

True

In [5]:
(3:) [4,7]

[3,4,7]

In [6]:
(++ "\n") "Hello" -- adds a new line char at the end of a string

"Hello\n"

In [7]:
("\n" ++) "Hello" -- adds a new line char at the beginning of a string

"\nHello"

In [8]:
filter (>3) [1,5,2,6]

[5,6]

In [9]:
map (`div` 2) [2,4,6] 

[1,2,3]

## Currying
Functions in Haskell are curried. It means that a function takes one parameter and returns another function which requires the remaining arguments.  
So we can say that **every function in Haskell takes exactly one argument**.

`func :: Int -> Int -> Int` is shorthand for `func :: Int -> (Int -> Int)`  
Therefore we can apply func to an integer.  

(func 3) :: Int -> Int

(func 3) 5 :: Int

In general: `func :: t1 -> t2 -> .. -> tn -> t` is actually `func :: t1 -> (t2 -> (.. -> (tn -> t)..))`

Note: function application is left associative and arrows in the signature are right associative.
- f a b is equal to (f a) b but not equal to f (a b).
- a -> b -> c is equal to a -> (b -> c) but not equal to (a -> b) -> c

`func a b .. z` is evaluated as `(..((func a) b).. z)`

### Currying vs Uncurrying
**Curried version** of sum function:
sum :: Integer -> Integer -> Integer
sum a b = a + b

**Uncurried version** of sum function:
sum :: (Integer, Integer) -> Integer
sum (a,b) = a + b

The curried version is what we would normally implement in Haskell.  

The uncurried version is similar to what we have in most of the programming languages. Uncurried version does not allow partial application since it already expects a single argument.

**Note: Arrows are NOT associative!**  
For example: `fun :: (Int -> Int) -> Int` expects a function, which takes an int and returns an int, as parameter

### Conversion between Curried and Uncurried Form

**curry** and **uncurry** functions in prelude can be used.
```Haskell
curry :: ((a, b) -> c) -> (a -> b -> c)
curry g x y = g (x,y)```  

```Haskell
uncurry :: (a -> b -> c) -> ((a,b) -> c)
uncurry g (a,b) = g a b```

### Function Composition (.)
f . g (x) = f (g x) 

What is the type of "."? 

Assume type of x is a.  
Then g takes an argument of type a and returns a value of type b  
f takes this argument of type b and returns a value of type c   

Let's write the signature of "function ."  
```Haskell 
(.) :: (b -> c) -> (a -> b) -> a -> c
(.) f g x = f (g x)
```

**(b -> c)** comes from the 1st argument, which is function f  
**(a -> b)** comes from the 2nd argument, which is function g  
a comes from the 3rd argument which is x  
c is the type of the return value (since f returns a value of type c, the return value of . is also c

*Note that f . g is NOT equal to g . f*

In [10]:
-- Let's compose not and even function in order to produce odd function
odd' :: Integer -> Bool
odd' = not . even 

odd' 13
odd' 14

True

False

In [11]:
-- function that returns the second element in a list
sec :: [a] -> a
sec = head . tail

sec [1,2,3,4,5]

2

#### Function Application via $  

In order to make the code more readable, we can eliminate parantheses via $.  

For instance,  
`sum ( filter odd (map (floor . sqrt) [1 .. 100]))`  
can be written as  
`sum $ filter odd $ map (floor . sqrt) [1 .. 100]`

In [12]:
zipWith ($) [sum, product] [[1, 2], [5, 10]] 

-- this does the following: [sum $ [1,2], product $ [5, 10]]

[3,50]